In [1]:
import os
import h5py
import pandas as pd
import time
import copy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
from random import randint
from random import shuffle

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader

%matplotlib inline
plt.ion()   # interactive mode

/opt/applications/python/3.6.3/gnu/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
PATH = "datasets/"
trainHdf5 = f'{PATH}Chong_train_set.hdf5'
traindata = h5py.File(trainHdf5, 'r')
valHdf5 = f'{PATH}Chong_valid_set.hdf5'
valdata = h5py.File(valHdf5, 'r')
testHdf5 = f'{PATH}Chong_test_set.hdf5'
testdata = h5py.File(testHdf5, 'r')

In [3]:
class ChongSet(Dataset):
    
    def __init__(self, hdf5, transform=None):
        classes = sorted(list(set([y for y in hdf5['label_names']])))
        class_to_idx = {x: classes.index(x) for x in classes}
        
        imgs = hdf5['data1']
        labels = hdf5['label_names']
        self.transform = transform
        self.classes = classes
        self.class_to_idx = class_to_idx
        self.imgs = imgs
        self.labels = labels
        self.data = []
        
        for i in range(len(imgs)):
            tmp = (np.array(imgs[i,:]).reshape(2,64,64).T, labels[i])
            self.data.append(tmp)
  
        
    def __len__(self):
        return(self.imgs.shape[0])
    
    def __getitem__(self, idx):
        img, cls = self.data[idx]
        cls = self.class_to_idx[cls]

        if self.transform:
            img = self.transform(img)
       
        return(img, cls)

In [4]:
data_transforms = {
    'train': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5], [0.5, 0.5])
    ]),
    'val':transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5], [0.5, 0.5])
    ]),
    'test':transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5], [0.5, 0.5])
    ]),
}

In [5]:
image_datasets = {'train': ChongSet(traindata, transform = data_transforms['train'] ), 'val': ChongSet(valdata, transform = data_transforms['val']), 'test': ChongSet(valdata, transform = data_transforms['test'])}
image_datasets

{'test': <__main__.ChongSet at 0x7fd55c744860>,
 'train': <__main__.ChongSet at 0x7fd55c745f98>,
 'val': <__main__.ChongSet at 0x7fd5d8130278>}

In [7]:
dataloaders = {'train':torch.utils.data.DataLoader(image_datasets['train'], batch_size=bs, shuffle=True, num_workers=4),
               'val': torch.utils.data.DataLoader(image_datasets['val'], batch_size=16, shuffle=False, num_workers=4),
               'test': torch.utils.data.DataLoader(image_datasets['test'], batch_size=16, shuffle=False,num_workers=4)}
dataloaders

{'test': <torch.utils.data.dataloader.DataLoader at 0x7fd55beaef28>,
 'train': <torch.utils.data.dataloader.DataLoader at 0x7fd55beaedd8>,
 'val': <torch.utils.data.dataloader.DataLoader at 0x7fd55beaee80>}

In [6]:
bs =64

In [8]:
nz=100
ngf=64
ndf=64

In [9]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [10]:
class _netG(nn.Module):
    def __init__(self):
        super(_netG, self).__init__()
        self.main = nn.Sequential(
            #input noise with dimension z
            nn.ConvTranspose2d(nz, ngf*8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf*8),
            nn.ReLU(True),
            #state size:(ngf*8) * 4 * 4
            nn.ConvTranspose2d(ngf*8, ngf*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*4),
            nn.ReLU(True),
            #state size: (ngf*4) *8*8
            nn.ConvTranspose2d(ngf*4, ngf*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*2),
            nn.ReLU(True),
            #state size: (ngf*2) * 16 * 16
            nn.ConvTranspose2d(ngf*2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            #state size: (ngf) * 32 * 32
            nn.ConvTranspose2d(ngf, 2, 4, 2, 1, bias=False),
            nn.Tanh()
        )
    def forward(self, input):
        output = self.main(input)
        return(output)
    

In [11]:
netG = _netG()
netG.apply(weights_init)

_netG(
  (main): Sequential(
    (0): ConvTranspose2d (100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU(inplace)
    (3): ConvTranspose2d (512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (5): ReLU(inplace)
    (6): ConvTranspose2d (256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (8): ReLU(inplace)
    (9): ConvTranspose2d (128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (11): ReLU(inplace)
    (12): ConvTranspose2d (64, 2, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)

In [12]:
class _netD(nn.Module):
    def __init__(self, dropoutrate):
        super(_netD, self).__init__()
        self.main = nn.Sequential(
            #input 2x64x64
            nn.Conv2d(2, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            
            #2*32*32
            nn.Conv2d(ndf, ndf*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf*2),
            nn.LeakyReLU(0.2, inplace=True),
            # 2*16*16
            nn.Conv2d(ndf*2, ndf*4, 4 ,2 ,1, bias=False),
            nn.BatchNorm2d(ndf*4),
            nn.LeakyReLU(0.2, inplace=True),
            
            # 2*8*8
            nn.Conv2d(ndf*4, ndf*8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf*8),
            nn.LeakyReLU(0.2, inplace=True),
            # 2*4*4
            nn.Conv2d(ndf*8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
            
            
        )
    def forward(self,input):
        output=self.main(input)
        #return output
        return output.view(-1,1).squeeze(1)

In [13]:
netD = _netD(0)
netD.apply(weights_init)

_netD(
  (main): Sequential(
    (0): Conv2d (2, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(0.2, inplace)
    (2): Conv2d (64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (4): LeakyReLU(0.2, inplace)
    (5): Conv2d (128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (7): LeakyReLU(0.2, inplace)
    (8): Conv2d (256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
    (10): LeakyReLU(0.2, inplace)
    (11): Conv2d (512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)

In [22]:
# ***************

In [14]:
criterion = nn.BCELoss()

In [15]:
input = torch.FloatTensor(bs, 2, 64, 64)
noise = torch.FloatTensor(bs, 100, 1, 1)
fixed_noise = torch.FloatTensor(bs, 100,1,1).normal_(0,1)
label = torch.FloatTensor(bs)
real_label=1
fake_label=0

In [16]:
netD.cuda()
netG.cuda()
criterion.cuda()
input,label = input.cuda(), label.cuda()
noise, fixed_noise = noise.cuda(), fixed_noise.cuda()

In [17]:
fixed_noise = Variable(fixed_noise)

In [18]:
optimizerD = optim.Adam(netD.parameters(), lr=0.0005, betas=(0.5,0.999))
optimizerG = optim.Adam(netG.parameters(), lr = 0.0005, betas = (0.5, 0.999))

In [20]:
for epoch in range(25):
    for i, data in enumerate(dataloaders['train']):
        netD.zero_grad()
        real_cpu, _ = data
        real_cpu = real_cpu.cuda()
        batch_size = real_cpu.size(0)
        input.resize_as_(real_cpu).copy_(real_cpu)
        label.resize_(batch_size).fill_(real_label)
        inputv = Variable(input)
        labelv = Variable(label)
        
        output = netD(inputv)
        errD_real = criterion(output, labelv)
        errD_real.backward()
        D_x = output.data.mean()
        
        noise.resize_(batch_size, nz, 1, 1).normal_(0,1)
        noisev=Variable(noise)
        fake=netG(noisev)
        labelv=Variable(label.fill_(fake_label))
        output = netD(fake.detach())
        errD_fake = criterion(output, labelv)
        errD_fake.backward()
        D_G_z1 = output.data.mean()
        errD = errD_real + errD_fake
        optimizerD.step()
        
        netG.zero_grad()
        labelv = Variable(label.fill_(real_label))
        output=netD(fake)
        errG = criterion(output,labelv)
        errG.backward()
        D_G_z2 = output.data.mean()
        optimizerG.step()
        
        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f' % (epoch, 100, i, len(dataloaders['train']), errD.data[0], errG.data[0], D_x, D_G_z1, D_G_z2))
        if i % 100 == 0:
            rdir = '%s/real_samples.png' % 'ganresult2'
            view_result(real_cpu, rdir)
            fake = netG(fixed_noise)
            fdir = '%s/fake_samples_epoch_%03d.png' %('ganresult2', epoch)
            view_result(fake.data, fdir)
    torch.save(netG.state_dict(), '%s/netG_epoch_%d.pth' % ('ganresult2', epoch))
    torch.save(netD.state_dict(), '%s/netD_epoch_%d.pth' % ('ganresult2', epoch))

[0/100][0/342] Loss_D: 1.0514 Loss_G: 11.4680 D(x): 0.6363 D(G(z)): 0.3851 / 0.0000
[0/100][1/342] Loss_D: 2.3904 Loss_G: 16.8777 D(x): 1.0000 D(G(z)): 0.8548 / 0.0000
[0/100][2/342] Loss_D: 0.0333 Loss_G: 11.8918 D(x): 0.9995 D(G(z)): 0.0313 / 0.0000
[0/100][3/342] Loss_D: 2.2758 Loss_G: 22.4571 D(x): 1.0000 D(G(z)): 0.8067 / 0.0000
[0/100][4/342] Loss_D: 0.5749 Loss_G: 21.2372 D(x): 0.9413 D(G(z)): 0.0000 / 0.0000
[0/100][5/342] Loss_D: 0.1342 Loss_G: 10.2804 D(x): 0.9454 D(G(z)): 0.0011 / 0.0001
[0/100][6/342] Loss_D: 3.4090 Loss_G: 27.0969 D(x): 0.9999 D(G(z)): 0.9436 / 0.0000
[0/100][7/342] Loss_D: 0.3046 Loss_G: 27.5831 D(x): 0.8750 D(G(z)): 0.0000 / 0.0000
[0/100][8/342] Loss_D: 0.0002 Loss_G: 27.6267 D(x): 0.9998 D(G(z)): 0.0000 / 0.0000
[0/100][9/342] Loss_D: 0.0006 Loss_G: 27.6297 D(x): 0.9994 D(G(z)): 0.0000 / 0.0000
[0/100][10/342] Loss_D: 0.0000 Loss_G: 27.6302 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[0/100][11/342] Loss_D: 0.0000 Loss_G: 27.6276 D(x): 1.0000 D(G(z)): 0.0000

[0/100][100/342] Loss_D: 0.9171 Loss_G: 4.2604 D(x): 0.9437 D(G(z)): 0.5368 / 0.0238
[0/100][101/342] Loss_D: 0.4774 Loss_G: 3.4006 D(x): 0.7474 D(G(z)): 0.1505 / 0.0459
[0/100][102/342] Loss_D: 1.9330 Loss_G: 0.0583 D(x): 0.2468 D(G(z)): 0.2474 / 0.9437
[0/100][103/342] Loss_D: 3.3612 Loss_G: 3.7547 D(x): 0.9733 D(G(z)): 0.9592 / 0.0303
[0/100][104/342] Loss_D: 1.6680 Loss_G: 1.2638 D(x): 0.2500 D(G(z)): 0.0751 / 0.3055
[0/100][105/342] Loss_D: 1.2276 Loss_G: 1.7741 D(x): 0.7034 D(G(z)): 0.5475 / 0.1865
[0/100][106/342] Loss_D: 1.0855 Loss_G: 1.3347 D(x): 0.5348 D(G(z)): 0.3035 / 0.2946
[0/100][107/342] Loss_D: 1.2293 Loss_G: 2.8002 D(x): 0.7287 D(G(z)): 0.5570 / 0.0732
[0/100][108/342] Loss_D: 1.4319 Loss_G: 1.3794 D(x): 0.4500 D(G(z)): 0.3668 / 0.2768
[0/100][109/342] Loss_D: 1.8298 Loss_G: 3.3796 D(x): 0.6101 D(G(z)): 0.6902 / 0.0542
[0/100][110/342] Loss_D: 2.4169 Loss_G: 0.3113 D(x): 0.1740 D(G(z)): 0.1555 / 0.7529
[0/100][111/342] Loss_D: 2.5638 Loss_G: 4.4828 D(x): 0.9058 D(G(z

[0/100][198/342] Loss_D: 1.1896 Loss_G: 0.2335 D(x): 0.4176 D(G(z)): 0.1925 / 0.7998
[0/100][199/342] Loss_D: 2.3023 Loss_G: 4.5066 D(x): 0.9582 D(G(z)): 0.8658 / 0.0126
[0/100][200/342] Loss_D: 0.5053 Loss_G: 3.3349 D(x): 0.7064 D(G(z)): 0.1283 / 0.0386
[0/100][201/342] Loss_D: 0.8562 Loss_G: 2.2459 D(x): 0.5651 D(G(z)): 0.1995 / 0.1282
[0/100][202/342] Loss_D: 2.9276 Loss_G: 0.1384 D(x): 0.0904 D(G(z)): 0.2429 / 0.8759
[0/100][203/342] Loss_D: 3.0298 Loss_G: 4.8479 D(x): 0.8684 D(G(z)): 0.9282 / 0.0899
[0/100][204/342] Loss_D: 1.3631 Loss_G: 4.0696 D(x): 0.6188 D(G(z)): 0.2061 / 0.0747
[0/100][205/342] Loss_D: 1.4502 Loss_G: 0.7501 D(x): 0.4346 D(G(z)): 0.2561 / 0.5342
[0/100][206/342] Loss_D: 1.8595 Loss_G: 3.1158 D(x): 0.8836 D(G(z)): 0.7640 / 0.0756
[0/100][207/342] Loss_D: 1.7676 Loss_G: 1.2195 D(x): 0.2495 D(G(z)): 0.1027 / 0.4616
[0/100][208/342] Loss_D: 1.5903 Loss_G: 2.2088 D(x): 0.8376 D(G(z)): 0.6111 / 0.1638
[0/100][209/342] Loss_D: 1.0660 Loss_G: 1.8698 D(x): 0.5352 D(G(z

[0/100][295/342] Loss_D: 0.9601 Loss_G: 2.6338 D(x): 0.6961 D(G(z)): 0.3882 / 0.0825
[0/100][296/342] Loss_D: 1.3575 Loss_G: 0.7906 D(x): 0.3913 D(G(z)): 0.2522 / 0.5005
[0/100][297/342] Loss_D: 1.4011 Loss_G: 2.2807 D(x): 0.8632 D(G(z)): 0.6645 / 0.1590
[0/100][298/342] Loss_D: 1.2244 Loss_G: 1.5880 D(x): 0.4465 D(G(z)): 0.1795 / 0.2510
[0/100][299/342] Loss_D: 1.2784 Loss_G: 2.3716 D(x): 0.8392 D(G(z)): 0.5981 / 0.1405
[0/100][300/342] Loss_D: 1.3431 Loss_G: 1.1451 D(x): 0.4320 D(G(z)): 0.2971 / 0.3740
[0/100][301/342] Loss_D: 1.3576 Loss_G: 2.2318 D(x): 0.6546 D(G(z)): 0.5218 / 0.1650
[0/100][302/342] Loss_D: 1.0386 Loss_G: 2.6774 D(x): 0.6152 D(G(z)): 0.3251 / 0.1174
[0/100][303/342] Loss_D: 1.1973 Loss_G: 1.2748 D(x): 0.4826 D(G(z)): 0.2810 / 0.3184
[0/100][304/342] Loss_D: 1.4320 Loss_G: 1.5876 D(x): 0.5970 D(G(z)): 0.5070 / 0.2419
[0/100][305/342] Loss_D: 1.4216 Loss_G: 1.6068 D(x): 0.5402 D(G(z)): 0.4680 / 0.2324
[0/100][306/342] Loss_D: 0.9485 Loss_G: 1.9267 D(x): 0.5975 D(G(z

[1/100][52/342] Loss_D: 1.5339 Loss_G: 1.0236 D(x): 0.4234 D(G(z)): 0.3610 / 0.4214
[1/100][53/342] Loss_D: 1.0345 Loss_G: 2.9592 D(x): 0.8246 D(G(z)): 0.5030 / 0.0729
[1/100][54/342] Loss_D: 1.0356 Loss_G: 1.2905 D(x): 0.4793 D(G(z)): 0.1837 / 0.3218
[1/100][55/342] Loss_D: 1.6069 Loss_G: 3.6047 D(x): 0.8753 D(G(z)): 0.7210 / 0.0397
[1/100][56/342] Loss_D: 2.2326 Loss_G: 0.4176 D(x): 0.1842 D(G(z)): 0.2227 / 0.6976
[1/100][57/342] Loss_D: 1.8443 Loss_G: 3.6710 D(x): 0.8636 D(G(z)): 0.7744 / 0.0324
[1/100][58/342] Loss_D: 1.5748 Loss_G: 1.9499 D(x): 0.3309 D(G(z)): 0.1972 / 0.1923
[1/100][59/342] Loss_D: 1.3252 Loss_G: 1.2787 D(x): 0.6541 D(G(z)): 0.4748 / 0.3184
[1/100][60/342] Loss_D: 1.2166 Loss_G: 1.6845 D(x): 0.5966 D(G(z)): 0.4203 / 0.2155
[1/100][61/342] Loss_D: 0.9595 Loss_G: 1.4586 D(x): 0.5608 D(G(z)): 0.2528 / 0.2778
[1/100][62/342] Loss_D: 1.6109 Loss_G: 1.7973 D(x): 0.5511 D(G(z)): 0.5710 / 0.2042
[1/100][63/342] Loss_D: 1.2096 Loss_G: 2.0607 D(x): 0.6045 D(G(z)): 0.4316 /

...

[98/100][222/342] Loss_D: 0.3347 Loss_G: 3.4835 D(x): 0.8037 D(G(z)): 0.0611 / 0.0577
[98/100][223/342] Loss_D: 0.7744 Loss_G: 7.8535 D(x): 0.8858 D(G(z)): 0.3500 / 0.0011
[98/100][224/342] Loss_D: 0.6293 Loss_G: 4.1267 D(x): 0.6390 D(G(z)): 0.0226 / 0.0472
[98/100][225/342] Loss_D: 0.8763 Loss_G: 8.8995 D(x): 0.9834 D(G(z)): 0.4472 / 0.0007
[98/100][226/342] Loss_D: 0.9626 Loss_G: 4.1487 D(x): 0.5541 D(G(z)): 0.0280 / 0.0475
[98/100][227/342] Loss_D: 1.0941 Loss_G: 10.2498 D(x): 0.9883 D(G(z)): 0.5232 / 0.0002
[98/100][228/342] Loss_D: 1.6985 Loss_G: 2.8116 D(x): 0.3468 D(G(z)): 0.0065 / 0.1843
[98/100][229/342] Loss_D: 1.7919 Loss_G: 9.1958 D(x): 0.9862 D(G(z)): 0.6483 / 0.0007
[98/100][230/342] Loss_D: 1.3184 Loss_G: 2.0214 D(x): 0.4652 D(G(z)): 0.0549 / 0.2773
[98/100][231/342] Loss_D: 0.4394 Loss_G: 5.6097 D(x): 0.9530 D(G(z)): 0.2136 / 0.0127
[98/100][232/342] Loss_D: 0.2119 Loss_G: 5.8959 D(x): 0.9922 D(G(z)): 0.1362 / 0.0076
[98/100][233/342] Loss_D: 0.7616 Loss_G: 4.9548 D(x):

[98/100][319/342] Loss_D: 0.4196 Loss_G: 5.9078 D(x): 0.9350 D(G(z)): 0.2246 / 0.0055
[98/100][320/342] Loss_D: 0.3847 Loss_G: 8.8118 D(x): 0.7336 D(G(z)): 0.0041 / 0.0013
[98/100][321/342] Loss_D: 0.3994 Loss_G: 3.7540 D(x): 0.8539 D(G(z)): 0.1112 / 0.0606
[98/100][322/342] Loss_D: 0.1062 Loss_G: 4.3200 D(x): 0.9749 D(G(z)): 0.0537 / 0.0289
[98/100][323/342] Loss_D: 0.6206 Loss_G: 6.4008 D(x): 0.9913 D(G(z)): 0.3485 / 0.0043
[98/100][324/342] Loss_D: 0.2876 Loss_G: 4.9000 D(x): 0.7930 D(G(z)): 0.0185 / 0.0168
[98/100][325/342] Loss_D: 0.5133 Loss_G: 3.7533 D(x): 0.7476 D(G(z)): 0.0593 / 0.0606
[98/100][326/342] Loss_D: 0.1352 Loss_G: 3.7882 D(x): 0.9772 D(G(z)): 0.0917 / 0.0435
[98/100][327/342] Loss_D: 0.2710 Loss_G: 5.6478 D(x): 0.9883 D(G(z)): 0.1541 / 0.0100
[98/100][328/342] Loss_D: 0.1081 Loss_G: 5.8392 D(x): 0.9718 D(G(z)): 0.0675 / 0.0116
[98/100][329/342] Loss_D: 0.1257 Loss_G: 5.3247 D(x): 0.9399 D(G(z)): 0.0530 / 0.0174
[98/100][330/342] Loss_D: 0.1005 Loss_G: 5.4710 D(x): 

[99/100][74/342] Loss_D: 0.4319 Loss_G: 4.6040 D(x): 0.8840 D(G(z)): 0.2123 / 0.0313
[99/100][75/342] Loss_D: 0.2033 Loss_G: 4.8225 D(x): 0.9115 D(G(z)): 0.0812 / 0.0212
[99/100][76/342] Loss_D: 0.2353 Loss_G: 5.1028 D(x): 0.8280 D(G(z)): 0.0063 / 0.0161
[99/100][77/342] Loss_D: 0.1565 Loss_G: 4.1845 D(x): 0.8967 D(G(z)): 0.0293 / 0.0347
[99/100][78/342] Loss_D: 0.1843 Loss_G: 4.4040 D(x): 0.9861 D(G(z)): 0.1339 / 0.0225
[99/100][79/342] Loss_D: 0.4526 Loss_G: 6.5029 D(x): 0.9848 D(G(z)): 0.2846 / 0.0046
[99/100][80/342] Loss_D: 1.0066 Loss_G: 2.1933 D(x): 0.4644 D(G(z)): 0.0324 / 0.2607
[99/100][81/342] Loss_D: 0.8907 Loss_G: 7.0893 D(x): 0.9898 D(G(z)): 0.4443 / 0.0026
[99/100][82/342] Loss_D: 0.4067 Loss_G: 4.8735 D(x): 0.8141 D(G(z)): 0.1118 / 0.0219
[99/100][83/342] Loss_D: 0.3926 Loss_G: 3.4433 D(x): 0.7974 D(G(z)): 0.0365 / 0.0837
[99/100][84/342] Loss_D: 0.1804 Loss_G: 4.4169 D(x): 0.9633 D(G(z)): 0.1111 / 0.0369
[99/100][85/342] Loss_D: 0.0937 Loss_G: 5.5469 D(x): 0.9286 D(G(z

[99/100][171/342] Loss_D: 0.3506 Loss_G: 4.2427 D(x): 0.8045 D(G(z)): 0.0777 / 0.0413
[99/100][172/342] Loss_D: 0.2021 Loss_G: 4.5666 D(x): 0.9574 D(G(z)): 0.1126 / 0.0266
[99/100][173/342] Loss_D: 0.4788 Loss_G: 6.0882 D(x): 0.9055 D(G(z)): 0.2600 / 0.0068
[99/100][174/342] Loss_D: 0.2245 Loss_G: 5.8269 D(x): 0.8288 D(G(z)): 0.0045 / 0.0074
[99/100][175/342] Loss_D: 0.3082 Loss_G: 2.9472 D(x): 0.8004 D(G(z)): 0.0440 / 0.1040
[99/100][176/342] Loss_D: 0.1979 Loss_G: 4.4672 D(x): 0.9741 D(G(z)): 0.1319 / 0.0244
[99/100][177/342] Loss_D: 0.0890 Loss_G: 5.2210 D(x): 0.9867 D(G(z)): 0.0680 / 0.0122
[99/100][178/342] Loss_D: 0.2931 Loss_G: 3.8107 D(x): 0.8665 D(G(z)): 0.1139 / 0.0434
[99/100][179/342] Loss_D: 0.3909 Loss_G: 3.4693 D(x): 0.8055 D(G(z)): 0.1040 / 0.0787
[99/100][180/342] Loss_D: 0.2057 Loss_G: 3.9873 D(x): 0.9034 D(G(z)): 0.0806 / 0.0349
[99/100][181/342] Loss_D: 0.2163 Loss_G: 4.7532 D(x): 0.9317 D(G(z)): 0.0992 / 0.0244
[99/100][182/342] Loss_D: 0.2796 Loss_G: 5.8579 D(x): 

[99/100][268/342] Loss_D: 0.5703 Loss_G: 3.8130 D(x): 0.6621 D(G(z)): 0.0246 / 0.0682
[99/100][269/342] Loss_D: 0.4726 Loss_G: 2.9112 D(x): 0.8132 D(G(z)): 0.1726 / 0.1323
[99/100][270/342] Loss_D: 0.1084 Loss_G: 4.9613 D(x): 0.9607 D(G(z)): 0.0540 / 0.0220
[99/100][271/342] Loss_D: 0.1260 Loss_G: 4.7786 D(x): 0.9357 D(G(z)): 0.0376 / 0.0245
[99/100][272/342] Loss_D: 0.6723 Loss_G: 6.4436 D(x): 0.9906 D(G(z)): 0.3668 / 0.0038
[99/100][273/342] Loss_D: 0.3247 Loss_G: 5.2163 D(x): 0.8055 D(G(z)): 0.0292 / 0.0135
[99/100][274/342] Loss_D: 0.3383 Loss_G: 3.9967 D(x): 0.7675 D(G(z)): 0.0242 / 0.0497
[99/100][275/342] Loss_D: 0.1994 Loss_G: 3.2282 D(x): 0.8820 D(G(z)): 0.0466 / 0.0836
[99/100][276/342] Loss_D: 0.8534 Loss_G: 6.6550 D(x): 0.9816 D(G(z)): 0.4577 / 0.0030
[99/100][277/342] Loss_D: 0.2292 Loss_G: 6.0599 D(x): 0.8245 D(G(z)): 0.0120 / 0.0065
[99/100][278/342] Loss_D: 0.1648 Loss_G: 5.8833 D(x): 0.8626 D(G(z)): 0.0024 / 0.0100
[99/100][279/342] Loss_D: 0.2956 Loss_G: 5.0508 D(x): 

In [19]:
def view_result(data, fdir):
    idx = np.arange(64)
    fig, axes = plt.subplots(8, 8, sharex=True,sharey=True, figsize=(15,15))
    for ii, ax in zip(idx, axes.flatten()):
        ax.imshow(data[ii][1,:,:],cmap=plt.cm.gist_gray,interpolation='bilinear')
        ax.imshow(data[ii][0,:,:],cmap=plt.cm.viridis, alpha=0.5,  interpolation='sinc')
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
    plt.subplots_adjust(wspace=0, hspace=0)
    fig.savefig(fdir)
    plt.close(fig)